## 00. Study previous assessments

## 0. Hey! Remember to use the tests!!!!

When you clone the assessment repo and you are in the final-assessment-files directory, make sure unittests works by entering make test in your terminal.

## 1. SQL

### 1a. readychef

If you still have the readychef database, you can access it by entering psql readychef in your terminal.

Write queries to find the following:

1. total meals bought by each user (including users who bought zero meals)
   
   
   this query gets total meals for every user but only for users who bought at least 1 meal

    SELECT userid, COUNT(*) as ct 
        FROM events 
        WHERE event='bought' 
        GROUP BY userid

    to get the 0-meal users, we LEFT JOIN the user column from "users" with the previous table, then use COALESCE to convert the NULL values to 0

    SELECT u.userid, COALESCE(c.ct,0) AS total_meals_bought FROM users AS u
    LEFT JOIN (
    SELECT userid, COUNT(*) as ct 
    FROM events 
    WHERE event='bought' 
    GROUP BY userid ) as c 
    ON u.userid = c.userid
    ORDER BY u.userid
    LIMIT 20;
    

2. Total money spent by each user
    
    SELECT u.userid, COALESCE(s.spent,0) AS total_spent FROM users AS u
    LEFT JOIN (
            SELECT userid, SUM(m.price) as spent 
            FROM events AS e
            JOIN meals AS m
             ON e.meal_id = m.meal_id
            WHERE event='bought' 
            GROUP BY userid ) as s
            ON u.userid = s.userid
    ORDER BY u.userid
    LIMIT 20;

3. Total visits from each user

    SELECT u.userid, COALESCE(v.ct,0) AS total_visits FROM users AS u
    LEFT JOIN (
            SELECT userid, COUNT(*) as ct 
            FROM visits 
            GROUP BY userid ) as v 
            ON u.userid = v.userid
    ORDER BY u.userid
    LIMIT 20;
    
4. Average visits per month from each user


WITH user_life AS
(
    SELECT userid,
      ((DATE_PART('year', a.max_dt) - DATE_PART('year', a.min_dt)) * 12 +
      (DATE_PART('month', a.max_dt) - DATE_PART('month', a.min_dt)) + 1) AS user_months
     FROM
        (
        SELECT userid, MIN(dt) as min_dt, MAX(dt) AS max_dt 
        FROM visits GROUP BY userid
        ) AS a
),
total AS
(
    SELECT u.userid, COALESCE(v.ct,0) AS total_visits FROM users AS u
    LEFT JOIN (
            SELECT userid, COUNT(*) as ct 
            FROM visits 
            GROUP BY userid ) as v 
         ON u.userid = v.userid
)
SELECT uu.userid, ROUND(tt.total_visits/uu.user_months :: numeric, 2)
FROM user_life AS uu
JOIN total AS tt
ON uu.userid = tt.userid
LIMIT 10;


select userid, date_part('month', dt) from visits group by userid, date_part('month', dt) limit 10;

### 1b. another table

i) Return the city with the highest population growth. (Highest net of people who currently live there minus people who used to live there)
CREATE TABLE inward_migration AS
  (SELECT current_city, COUNT(*) AS net_in
  FROM customers
  GROUP BY current_city)

  CREATE TABLE outward_migration AS
  (SELECT hometown, COUNT(*) AS net_out
  FROM customers
  GROUP BY hometown)

  SELECT a.current_city AS city,
    a.net_in - b.net_out AS net_immigration
  FROM inward_migration a
  JOIN outward_migration b
  ON a.current_city = b.hometown
  ORDER BY net_immigration DESC
  LIMIT 1
  
ii) Return pairs of "friends" (can be two columns or a tuple) that have both the same hometown and current city. Remove duplicates!
SELECT a.cust_id AS friend1, b.cust_id AS friend2
  FROM customers a
  JOIN customers b
  ON a.hometown = b.hometown
  AND a.current_city = b.current_city
  WHERE a.cust_id < b.cust_id


## 2. Web scraping

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
query = 'data scientist'

In [3]:
url = "http://www.indeed.com/jobs?q={}".format(query.replace(' ', '+'))
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [4]:
url


'http://www.indeed.com/jobs?q=data+scientist'

In [5]:
# find the number of results
# You can search by tag, attribute, or both

soup.find('div', id='searchCount').text

'\n        Page 1 of 23,570 jobs'

In [6]:
soup.find('div', id='searchCount')

<div id="searchCount">
        Page 1 of 23,570 jobs</div>

In [7]:
soup.find(id='searchCount')

<div id="searchCount">
        Page 1 of 23,570 jobs</div>

In [8]:
soup.find(id='searchCount').text

'\n        Page 1 of 23,570 jobs'

In [9]:
soup.find(id='searchCount').text.split()

['Page', '1', 'of', '23,570', 'jobs']

In [10]:
soup.find(id='searchCount').text.split()[-1]

'jobs'

In [16]:
import requests
from bs4 import BeautifulSoup

def number_of_jobs(query):
    '''
    INPUT: string
    OUTPUT: int

    Return the number of jobs on the indeed.com for the search query.
    '''

    url = "http://www.indeed.com/jobs?q={}".format(query.replace(' ', '+'))
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    search_count = soup.find('div', id='searchCount')
    return int(search_count.text.split()[-2].replace(',', ''))

In [17]:
number_of_jobs('data scientist')

23567

In [18]:
soup.find_all('div', class_='row result')

[<div class="row result" data-advn="5934651801106676" data-jk="7dee0a3754255bee" id="pj_7dee0a3754255bee">
 <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0DV4VMfmBJI2fADxaSmO4APG19pQrP8HAfsIrAnoAIqcE6KtGnsd-lg0hs2Usoksi-GyvPGYRgLzK4Qz5WOJcfBYXKdSJ_nir_NJlpEGdOAwaMLD2_H1rNmYJSfFU0_yJ_SxwbJrSmTlbhq7bCJcp8UzxwpUZ8mCu4BZeiQhSsaG1Y9zatOv_HQQ-MdzzNskKOgZKWeq3E3ROv2yU5UYToqhqOGBlP6qNfx1KE1Es-ymg5xALZErNm2-kQR5ZMFwlbqbLqi_vlRJjzYOcV7YezttM5JBNK3A6TnTMvGy6FrXp-1he-g4LpPSB-DecAy89i5Sf6vv_R1gGtA2l4SFzDZFvcdHjt330p_Jx6xLjl-pjhTOps_yYgn56fwOVYyO4NdZM4wuy1rV3cZ_BftnUqd4AHrasHbsLo9-9kZb_WTbvFCPmKuSNUiv5p8ZZlfK83ojr5uSV57MwENHk-VJBP3hpZJZxcEDQBUaa-8DY6fLIxHxcvkASHxvbrH9cAdqSr7P7hi5pgEISCM4AMPPnNOePUJ2MvaMaVE0im-kpKOnJKM22eKpKWnyO1rRvfGvdq7OwmrS-Kl64g-dhKz0qq61Xadv77UjAU=&amp;p=1&amp;sk=&amp;fvj=1" id="sja1" onclick="setRefineByCookie([]); sjoc('sja1',0); convCtr('SJ')" onmousedown="sjomd('sja1'); clk('sja1');" rel="noopener nofollow" target="_blank

## 3. Distributions

http://www.cs.elte.hu/~mesti/valszam/kepletek.pdf

Beta distribution:
$B(\alpha, \beta) $

$\alpha = 1 + (\#successes)$

$\beta = 1 + (\#failures)$

http://stats.stackexchange.com/a/47782

## 4. Hypothesis testing

https://github.com/gschool/DSI_Lectures/blob/master/ab-testing/tammy_lee/lecture.pdf